In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.2 MB/s eta 0:00:00


In [2]:
!pip install sentence_transformers

In [3]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sentence_transformers import SentenceTransformer
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features with SBERT embeddings
def process_features_sbert(input_file, text_column, numeric_features, model_name="all-MiniLM-L6-v2"):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Load SBERT model
    print("Loading SBERT model...")
    model = SentenceTransformer(model_name)

    # Generate SBERT embeddings for each text input
    print("Generating SBERT embeddings...")
    embeddings = model.encode(df[text_column].astype(str).tolist(), convert_to_numpy=True, show_progress_bar=True)
    print(f"SBERT embeddings shape: {embeddings.shape}")

    # Extract numeric features and combine with embeddings
    numeric_df = df[numeric_features]
    combined_features = np.hstack([embeddings, numeric_df.to_numpy()])

    return pd.DataFrame(combined_features), df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Define model functions
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'solver': ['liblinear', 'saga'], 'max_iter': [100, 200]}
    model = tune_model(LogisticRegression(), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'SVM': svm_model,
        'Logistic Regression': logistic_regression_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Process the features with SBERT
X, original_data = process_features_sbert(input_file, text_column, numeric_features)

# Prepare target variable with shifted labels
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models with hyperparameter tuning
run_all_models(X_train, y_train, X_test, y_test)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Mounted at /content/drive
Loading SBERT model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Best Params: {'kernel': 'linear', 'gamma': 'scale', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9307
Train Precision: 0.9304
Train Recall: 0.9307
Train F1 Score: 0.9295
Train Confusion Matrix:
[[ 30   1   1]
 [  2 109  21]
 [  0   8 304]]

Test Accuracy: 0.6585
Test Precision: 0.6708
Test Recall: 0.6585
Test F1 Score: 0.6639
Test Confusion Matrix:
[[  4   4   5]
 [  1  15  25]
 [  6  29 116]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=30. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'max_iter': 100, 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.8655
Train Precision: 0.8692
Train Recall: 0.8655
Train F1 Score: 0.8579
Train Confusion Matrix:
[[ 16   4  12]
 [  1  93  38]
 [  0   9 303]]

Test Accuracy: 0.6976
Test Precision: 0.6772
Test Recall: 0.6976
Test F1 Score: 0.6754
Test Confusion Matrix:
[[  2   5   6]
 [  1  10  30]
 [  0  20 131]]

Running Decision Tree model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': None}
Decision Tree Model Metrics:
Train Accuracy: 0.9622
Train Precision: 0.9618
Train Recall: 0.9622
Train F1 Score: 0.9619
Train Confusion Matrix:
[[ 26   2   4]
 [  3 126   3]
 [  1   5 306]]

Test Accuracy: 0.6098
Test Precision: 0.6504
Test Recall: 0.6098
Test F1 Score: 0.6266
Test Confusion Matrix:
[[  2   2   9]
 [  1  19  21]
 [ 15  32 104]]

Running Naive Bayes model...
Naive Bayes Model Metrics:
Train Accuracy: 0.5987
Train Precision: 0.7312
Train Recall: 0.5987
Train F1 Score: 0.6125
Train Confusion Matrix:
[[ 25   5   2]
 [ 11 103  18]
 [  9 146 157]]

Test Accuracy: 0.4634
Test Precision: 0.6796
Test Recall: 0.4634
Test F1 Score: 0.5062
Test Confusion Matrix:
[[ 5  7  1]
 [ 4 25 12]
 [ 6 80 65]]

